# Revised case normalization for Hirslanden Zürich 2019

Notes:

- Inserted 32 revised cases into the DB. 
- Previous number of matched cases: 55



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
                     'Hirslanden Klinik Zurich', '2019', 'Änderungen_MIX_Hirslanden_2019')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Klinik Zurich', year='2019', sheet='Änderungen_MIX_Hirslanden_2019')


In [3]:
columns_to_rename = dict(COLUMNS_TO_RENAME)
columns_to_rename.pop("admno")
columns_to_rename['fall nummer'] = CASE_ID_COL

revised_cases_df = normalize(file_info, columns_mapper = columns_to_rename)
revised_cases_df

2022-11-21 09:16:57.786 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 155 cases for Hirslanden Klinik Zurich 2019
2022-11-21 09:16:57.790 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:61 - 77/155 rows were deleted because contained NaNs
2022-11-21 09:16:57.792 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum abgabe               string
case_id                    object
patient_id                 object
kkik                       string
gender                     string
age_years                   int64
bfs_code                   string
duration_of_stay            int64
pflegetage neu             string
pccl                        int64
pccl neu                   string
old_pd                     string
primary_diagnosis          string
added_icds                 string
removed_icds               string
added_chops                string
removed_chops   

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
44,44107556,CEDB707C593C2DE7,M,79,4,44107556,T821,T821,M200,3,F18B,[L8932],[],[],[]
46,44110727,D5D63C950FB90094,W,77,42,44110727,T828,T828,M200,3,F34D,"[D62, I350]",[],[],"[380618:L:20190207, 380810:L:20190207, 884899:L:20190207, 0042::20190207, 397515:L:20190207, ...]"
48,44118028,1AA65D0D07A0DF94,M,85,9,44118028,M4806,M4806,M200,3,I10C,[G2090],[],[],[]
50,44119597,BB3A9FAED297483D,M,71,7,44119597,K660,K660,M000,3,G21B,[J841],[],[],[]
52,44125669,1C39D2D643D1989A,M,77,10,44125669,N304,N304,M200,3,L04C,[T820],[],[],[]
54,44135768,AF0C967A0B364BA0,M,64,20,44135768,M4727,M4727,M200,3,I95Z,[F058],[],[],[]
56,44139324,E0CA9A7527DE206E,M,65,3,44139324,I6710,I6710,M850,3,B20D,[I422],[],[],[]
58,44153256,1A6A155734AD077E,M,83,31,44153256,M511,M511,M200,3,I09D,[K521],[],[],[889714::20190613]
60,44156085,0FD90F4C5745F64B,M,73,31,44156085,T855,T855,M200,3,G70B,"[M6250, G8243, R470]",[G819],[],[]
62,44163377,73730420B5E0D968,W,75,11,44163377,M512,M512,M200,3,I10C,[G8229],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.dtypes

2022-11-21 09:17:17.111 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 15107 rows from the DB, for the hospital 'Hirslanden Klinik Zurich' in 2019


aimedic_id                 int64
case_id                   object
primary_diagnosis         object
secondary_diagnoses       object
primary_procedure         object
secondary_procedures      object
gender                    object
age_years                  int64
age_days                   int64
gestation_age              int64
duration_of_stay           int64
ventilation_hours          int64
grouper_admission_type    object
admission_date            object
admission_weight           int64
grouper_discharge_type    object
discharge_date            object
dtype: object

In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:17:18.226 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 32 cases ...
2022-11-21 09:17:19.325 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 32 cases into: 32 revisions, 314 diagnoses rows, 247 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,393249,H01A,5.00,5.00,4,2022-12-31
1,393971,F34C,3.50,3.50,4,2022-12-31
2,394239,H01A,5.00,5.00,4,2022-12-31
3,394542,B04A,3.88,3.88,4,2022-12-31
4,395176,L03Z,3.27,3.27,4,2022-12-31
5,395286,G64A,1.67,2.53,4,2022-12-31
6,395693,I06A,7.04,7.04,4,2022-12-31
7,395742,J02A,4.88,4.88,4,2022-12-31
8,397255,F34C,3.50,3.50,4,2022-12-31
9,397837,B02C,3.02,3.02,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:17:19.344 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 32 cases into the 'Revisions' table ...
2022-11-21 09:17:37.692 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 32 cases into the 'Revisions' table
2022-11-21 09:17:37.695 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 314 rows into the 'Diagnoses' table ...
2022-11-21 09:17:43.608 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 314 rows into the 'Diagnoses' table
2022-11-21 09:17:43.610 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 247 rows into the 'Procedures' table ...
2022-11-21 09:17:45.199 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 247 rows into the 'Procedures' table
